In [23]:
!pip uninstall numpy -v


^C


In [19]:
!pip install opencv-python
# !pip install tqdm
# !pip install pathlib
!pip install ultralytics

In [20]:
import os
import re
# import cv2
import yaml
import shutil
import random
from tqdm import tqdm
from pathlib import Path
from ultralytics import YOLO


ModuleNotFoundError: No module named 'ultralytics'

In [3]:
IMG_PATH = Path(r'./data/img_dataset')
ANNOTATION_PATH = Path(r'./data/annotation')
train_val_test = (0.8,0.1,0.1)

class_dict = {category_name: idx for idx, category_name in enumerate(os.listdir(IMG_PATH))}
print(class_dict)

{'Bicycle': 0, 'Boat': 1, 'Bottle': 2, 'Bus': 3, 'Car': 4, 'Cat': 5, 'Chair': 6, 'Cup': 7, 'Dog': 8, 'Motorbike': 9, 'People': 10, 'Table': 11}


In [ ]:
# def clean_file_names():
#     DATA_PATH = r'./data'
#     for root, _, file_names in os.walk(DATA_PATH):
#         if file_names == []:
#             continue
        
#         for file_name in file_names:
#             new_file_name = file_name.lower()
#             old_file_path = os.path.join(root, file_name)
#             file_path = os.path.join(root, new_file_name)
#             os.rename(old_file_path, file_path)

# clean_file_names()

In [6]:
def pascal_to_yolo(image_path):
    image_name = os.path.basename(image_path)
    image_dir = os.path.dirname(image_path).replace("\\", "/")
    file_title, _ = os.path.splitext(image_name)
    yolo_anno_path = image_dir.replace("images", "labels") + f"/{file_title}.txt"

    img = cv2.imread(image_path)
    img_height, img_width = img.shape[:2]

    sub_category = image_dir.split("/")[-1]
    path_from_base = os.path.join(sub_category, image_name)
    
    original_annotation_path = os.path.join(ANNOTATION_PATH, path_from_base + ".txt")
    with open(original_annotation_path, 'r') as anno_file:
        pascal_data = anno_file.readlines()[1:]

    pascal_num = list(map(lambda s: s.strip().split(" ")[:5], pascal_data))
    
    yolo_format = []
    for indv_bounding_coordinates in pascal_num:
        indv_bounding_coordinates[0] = class_dict[indv_bounding_coordinates[0]]
        indv_bounding_coordinates[1:] = list(map(int, indv_bounding_coordinates[1:]))
        print(indv_bounding_coordinates)

        class_category = indv_bounding_coordinates[0]
        x_ctr = (indv_bounding_coordinates[1] + indv_bounding_coordinates[3]/2) / img_width
        y_ctr = (indv_bounding_coordinates[2] + indv_bounding_coordinates[4]/2) / img_height
        width = (indv_bounding_coordinates[3]) / img_width
        height = (indv_bounding_coordinates[4]) / img_height
        yolo_format.append([class_category, x_ctr, y_ctr, width, height])
    
    os.makedirs(os.path.dirname(yolo_anno_path), exist_ok=True)
    with open(yolo_anno_path, 'w') as yolo_writer:
        for data in yolo_format:
            yolo_writer.write(f"{' '.join(list(map(str, data)))}\n")
    
    print(f"Converted {image_path} to {yolo_anno_path}")
    

In [61]:
testimg = r'C:\Users\Admin\Documents\MLGP\mlgp_full\data\img_dataset\Dog\2015_05337.jpeg'
print(cv2.imread(testimg).shape)

(2560, 1920, 3)


In [7]:
def generate_train_val_test(src_path, train_path, val_path, test_path, split_ratio, seed=None):
    if not os.path.exists(train_path):
        os.makedirs(train_path, exist_ok=True)
    if not os.path.exists(val_path):
        os.makedirs(val_path, exist_ok=True)
    if not os.path.exists(test_path):
        os.makedirs(test_path, exist_ok=True)

    if os.listdir(train_path) != []:
        shutil.rmtree(train_path)
        print(f"Removed existing train path: {train_path}")
    if os.listdir(val_path) != []:
        shutil.rmtree(val_path)
        print(f"Removed existing val path: {val_path}")
    if os.listdir(test_path) != []:
        shutil.rmtree(test_path)
        print(f"Removed existing test path: {test_path}")
    
    if seed != None:
        random.seed(seed)

    file_lists = []
    for (root, _, file_names) in os.walk(src_path):
        if file_names == []:
            continue

        sub_folder_name = os.path.basename(root)

        file_path_list = list(map(lambda path: os.path.join(sub_folder_name, path), file_names))

        random.shuffle(file_path_list)
        file_lists.append(file_path_list)

    for category_specific_list in file_lists:
        for file_name in category_specific_list:
            random_num = random.random()

            if random_num < split_ratio[0]:
                move_path = os.path.join(train_path, os.path.dirname(file_name))
            elif random_num < sum(split_ratio[:2]):
                move_path = os.path.join(val_path, os.path.dirname(file_name)) 
            else:
                move_path = os.path.join(test_path, os.path.dirname(file_name))

            os.makedirs(move_path, exist_ok=True)            
            original_path = os.path.join(src_path, file_name)
            new_name = os.path.join(move_path, os.path.basename(file_name))
            shutil.copy(original_path, move_path)

            pascal_to_yolo(new_name)
            
            print(original_path)
            print(new_name)


generate_train_val_test(
    src_path='data/img_dataset',
    train_path='data/train_val/images/train',
    val_path='data/train_val/images/val',
    test_path='data/test_data/images',
    split_ratio=train_val_test
    )

[0, 238, 424, 136, 88]
[10, 42, 369, 48, 137]
[4, 344, 391, 61, 48]
[4, 389, 389, 35, 61]
Converted data/train_val/images/val\Bicycle\2015_00189.jpg to data/train_val/labels/val/Bicycle/2015_00189.txt
data/img_dataset\Bicycle\2015_00189.jpg
data/train_val/images/val\Bicycle\2015_00189.jpg
[0, 131, 311, 109, 73]
Converted data/test_data/images\Bicycle\2015_00368.jpg to data/test_data/labels/Bicycle/2015_00368.txt
data/img_dataset\Bicycle\2015_00368.jpg
data/test_data/images\Bicycle\2015_00368.jpg
[0, 183, 175, 139, 114]
[10, 222, 102, 56, 141]
Converted data/train_val/images/train\Bicycle\2015_00140.jpg to data/train_val/labels/train/Bicycle/2015_00140.txt
data/img_dataset\Bicycle\2015_00140.jpg
data/train_val/images/train\Bicycle\2015_00140.jpg
[0, 248, 174, 128, 127]
[4, 30, 187, 87, 56]
[4, 525, 193, 64, 56]
[10, 125, 165, 36, 92]
[10, 176, 167, 44, 88]
[10, 460, 179, 48, 92]
Converted data/train_val/images/train\Bicycle\2015_00048.jpg to data/train_val/labels/train/Bicycle/2015_0004

In [137]:
def create_data_yaml(path='./'):
    data = {
        'path': path,
        'train': './data/train_data',
        'val': './data/val_data',
        'nc': len(class_dict),
        'names': list(class_dict.keys()),
    }

    with open('data.yaml', 'w') as f:
        yaml.dump(data, f, default_flow_style=False)

# Example usage
create_data_yaml()

In [55]:
# check for different file names for images and labels
root_data = r'.\data'
test_data = os.path.join(root_data, "test_data")
train_val = os.path.join(root_data, "train_val")


def check_file_names(folder): # test_data or train_val
    img_folder = os.path.join(folder, "images")
    label_folder = os.path.join(folder, "labels")
    train_img = os.path.join(img_folder, "train")
    val_img = os.path.join(img_folder, "val")
    train_label = os.path.join(label_folder, "train")
    val_label = os.path.join(label_folder, "val")

    for (root, dirname, filename) in os.walk(train_img):
        for objtype in dirname:
            obj_path = os.path.join(root, objtype)
            for (root, dirname, file_names) in os.walk(obj_path):
                for file_name in file_names:
                    file_name = os.path.splitext(file_name)[0]
                    label_name = os.path.join(train_label, objtype, file_name + ".txt")
                    if not os.path.exists(label_name):
                        print(f"Missing label file for {file_name} in {label_name}")

check_file_names(train_val)
check_file_names(test_data)

In [65]:
# check label files if width and height >1
def check_label_files(folder):

    label_folder = os.path.join(folder, "labels")
    for root, dirname, files in os.walk(label_folder):
        for file in files:
            label_path = os.path.join(root, file)
            with open(label_path, 'r') as f:
                lines = f.readlines()
                new_lines = []
                for line in lines:
                    values = list(map(float, line.strip().split()))
                    if values[1] > 1 or values[2] > 1 or values[3] > 1 or values[4] > 1:
                        values[1] = 1.0 if values[1] > 1 else values[1]
                        values[2] = 1.0 if values[2] > 1 else values[2]
                        values[3] = 1.0 if values[3] > 1 else values[3]
                        values[4] = 1.0 if values[4] > 1 else values[4]
                        print(f"Invalid bounding box in {label_path}: {line.strip()}")
                    if values[1] < 0 or values[2] < 0 or values[3] < 0 or values[4] < 0:
                        values[1] = 0.0 if values[1] < 0 else values[1]
                        values[2] = 0.0 if values[2] < 0 else values[2]
                        values[3] = 0.0 if values[3] < 0 else values[3]
                        values[4] = 0.0 if values[4] < 0 else values[4]
                        print(f"Invalid bounding box in {label_path}: {line.strip()}")
                    if len(values) != 5:
                        print(f"Invalid label format in {label_path}: {line.strip()}")
                    new_line = " ".join(str(v) for v in values) + "\n"
                    new_lines.append(new_line)
                # Overwrite the original file with the corrected content
                with open(label_path, 'w') as f:
                    f.writelines(new_lines)
                

check_label_files(train_val)
check_label_files(test_data)

Invalid bounding box in .\data\test_data\labels\Dog\2015_05337.txt: 8 0.6395833333333333 0.421875 1.075 0.6078125


In [138]:
with open("data.yaml") as f:
    cfg = yaml.safe_load(f)

print(cfg)

{'names': ['People', 'Motorbike', 'Cup', 'Bicycle', 'Chair', 'Boat', 'Table', 'Car', 'Bottle', 'Bus', 'Cat', 'Dog'], 'nc': 12, 'path': './', 'train': './data/train_data', 'val': './data/val_data'}


In [39]:
model = YOLO('./snapshots/yolov8n.pt')

print(model.info())

YOLOv8n summary: 129 layers, 3,157,200 parameters, 0 gradients, 8.9 GFLOPs
(129, 3157200, 0, 8.8575488)


In [40]:
config_settings = {
    # Output Config
    "project": "./YOLOv8_outputs/runs/train",
    "name": "yolov8_exp",

    # Train Config
    "epochs": 1000,
    "patience": 10,
    "batch": 15,
    "imgsz": 640,
    "save": True,
    "save_period": 1,
    "cache": False,
    "device": 0,
    "workers": 8,
    "project": "runs/train",
    "name": "yolov8_exp",
    "exist_ok": True,
    "pretrained": True,
    "deterministic": False,
    "classes":list(class_dict.keys()),
    "resume": True,
}


In [41]:
model.train(
    data="data.yaml",
    **config_settings
)


Ultralytics 8.3.107  Python-3.13.2 torch-2.6.0+cpu 


ValueError: Invalid CUDA 'device=0' requested. Use 'device=cpu' or pass valid CUDA device(s) if available, i.e. 'device=0' or 'device=0,1,2,3' for Multi-GPU.

torch.cuda.is_available(): False
torch.cuda.device_count(): 0
os.environ['CUDA_VISIBLE_DEVICES']: None
See https://pytorch.org/get-started/locally/ for up-to-date torch install instructions if no CUDA devices are seen by torch.
